In [1]:
class HMM:
    def forward(A,B,pi,N,M,sequence):
        '''
        Only Consider B as a matrix, indicating finite discrete outcomes.
        N: Number of states
        B: R^{N*M}
        A: R^{N*N}
        '''
        T=len(sequence)
        alpha=np.zeros([T,N])
        alpha[0]=pi*B[:,sequence[0]]
        t=1
        while True:
            if t==T:
                break
            alpha[t]=alpha[t-1]@A*B[:,sequence[t]]
            t+=1
        return(np.sum(alpha[T-1]))

In [2]:
import numpy as np
M=3
N=3
pi=np.array([.3,.3,.4])
A=np.array([[.2,.3,.5],[.1,.5,.4],[.6,.1,.3]])
B=np.array([[0.1,0.5,0.4],[0.2,0.4,0.4],[0.3,0.6,0.1]])
sequence=[0,1,2]
print(HMM.forward(A,B,pi,N,M,sequence))

0.030162
